# 📊 Churn Prediction Analytics Dashboard

This notebook visualizes real-time churn prediction analytics from the Kafka streaming pipeline.

**Data Sources:**
- RDS PostgreSQL Analytics Database
- Tables: `churn_predictions`, `churn_metrics_hourly`, `high_risk_customers`
- Views: Pre-aggregated analytics for dashboards

**Visualizations:**
1. Real-Time Dashboard (Last 24 Hours)
2. Geography-wise Churn Analysis
3. High-Risk Customer Alerts
4. Churn Trends Over Time
5. Model Performance Metrics
6. Feature Importance Analysis


## 📦 Setup: Import Libraries


In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import psycopg2
from datetime import datetime, timedelta
from dotenv import load_dotenv
import warnings

warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


## 🔌 Connect to RDS Analytics Database


In [2]:
# Load environment variables
project_root = os.path.dirname(os.getcwd())
env_path = os.path.join(project_root, '.env')
load_dotenv(env_path)

# RDS connection parameters
RDS_CONFIG = {
    'host': os.getenv('RDS_HOST'),
    'port': int(os.getenv('RDS_PORT', 5432)),
    'database': os.getenv('RDS_DB_NAME', 'analytics'),
    'user': os.getenv('RDS_USERNAME'),
    'password': os.getenv('RDS_PASSWORD'),
    'sslmode': 'require'
}

print(f"🔗 Connecting to RDS: {RDS_CONFIG['host']}")

# Create connection
try:
    conn = psycopg2.connect(**RDS_CONFIG)
    print(f"✅ Connected to analytics database: {RDS_CONFIG['database']}")
    
    # Test connection
    with conn.cursor() as cur:
        cur.execute("SELECT version();")
        db_version = cur.fetchone()[0]
        print(f"📊 Database: {db_version.split(',')[0]}")
        
        # Check available tables
        cur.execute("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
            ORDER BY table_name;
        """)
        tables = [row[0] for row in cur.fetchall()]
        print(f"📁 Available tables: {', '.join(tables)}")
        
except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise


🔗 Connecting to RDS: churn-pipeline-metadata-db.cbqsg4cugpeo.ap-south-1.rds.amazonaws.com
✅ Connected to analytics database: analytics
📊 Database: PostgreSQL 17.4 on aarch64-unknown-linux-gnu
📁 Available tables: churn_metrics_daily, churn_metrics_hourly, churn_predictions, churn_predictions_archive, high_risk_customers


## 📊 Helper Functions


In [3]:
def query_to_dataframe(query, conn):
    """Execute SQL query and return pandas DataFrame"""
    try:
        df = pd.read_sql_query(query, conn)
        return df
    except Exception as e:
        print(f"❌ Query failed: {e}")
        return pd.DataFrame()

def get_table_stats(table_name, conn):
    """Get basic statistics for a table"""
    query = f"""
        SELECT 
            COUNT(*) as total_rows,
            MIN(created_at) as oldest_record,
            MAX(created_at) as newest_record
        FROM {table_name};
    """
    return query_to_dataframe(query, conn)

print("✅ Helper functions defined")


✅ Helper functions defined


docker logs kafka-analytics --follow

In [4]:
# Query real-time dashboard view
query_realtime = """
    SELECT * FROM v_realtime_dashboard
    ORDER BY hour DESC
    LIMIT 24;
"""

df_realtime = query_to_dataframe(query_realtime, conn)

if not df_realtime.empty:
    print(f"📊 Real-time data: {len(df_realtime)} hourly records")
    print(f"📅 Date range: {df_realtime['hour'].min()} to {df_realtime['hour'].max()}")
    print(f"\n📊 Summary Statistics:")
    print(f"  • Total Predictions: {df_realtime['total_predictions'].sum():,.0f}")
    print(f"  • Total Churns: {df_realtime['churn_count'].sum():,.0f}")
    print(f"  • Average Churn Rate: {df_realtime['churn_rate'].mean():.2f}%")
    print(f"  • High-Risk Customers: {df_realtime['high_risk_count'].sum():,.0f}")
    
    # Display sample
    display(df_realtime.head())
else:
    print("⚠️ No real-time data available yet. Predictions will appear once consumer processes events.")


📊 Real-time data: 1 hourly records
📅 Date range: 2025-10-18 18:00:00 to 2025-10-18 18:00:00

📊 Summary Statistics:
  • Total Predictions: 1,859
  • Total Churns: 484
  • Average Churn Rate: 26.04%
  • High-Risk Customers: 507


,hour,total_predictions,churn_count,churn_rate,avg_risk_score,high_risk_count,latest_prediction
0,2025-10-18 18:00:00,1859,484,26.04,0.647,507,2025-10-18 18:48:49.321052


In [5]:
# Visualization: Churn Rate and Volume Over Time
if not df_realtime.empty:
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Prediction Volume Over Time', 'Churn Rate Over Time'),
        vertical_spacing=0.15
    )
    
    # Plot 1: Prediction volume
    fig.add_trace(
        go.Bar(
            x=df_realtime['hour'],
            y=df_realtime['total_predictions'],
            name='Total Predictions',
            marker_color='lightblue'
        ),
        row=1, col=1
    )
    
    # Plot 2: Churn rate
    fig.add_trace(
        go.Scatter(
            x=df_realtime['hour'],
            y=df_realtime['churn_rate'],
            name='Churn Rate (%)',
            mode='lines+markers',
            line=dict(color='red', width=3),
            marker=dict(size=8)
        ),
        row=2, col=1
    )
    
    fig.update_xaxes(title_text="Hour", row=2, col=1)
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="Churn Rate (%)", row=2, col=1)
    
    fig.update_layout(
        height=700,
        title_text="Real-Time Churn Prediction Analytics (Last 24 Hours)",
        showlegend=False
    )
    
    fig.show()
else:
    print("⏳ Waiting for prediction data...")


## 🌍 2. Geography-wise Churn Analysis


In [6]:
# Query geography analysis view
query_geography = """
    SELECT * FROM v_geography_churn
    ORDER BY churn_rate DESC;
"""

df_geography = query_to_dataframe(query_geography, conn)

if not df_geography.empty:
    print("🌍 Geography-wise Analysis:")
    display(df_geography)
    
    # Create visualizations
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Churn Rate by Geography', 'Customer Distribution'),
        specs=[[{'type': 'bar'}, {'type': 'pie'}]]
    )
    
    # Bar chart: Churn rates
    fig.add_trace(
        go.Bar(
            x=df_geography['geography'],
            y=df_geography['churn_rate'],
            text=df_geography['churn_rate'].round(2),
            textposition='auto',
            marker_color=['red' if x > 25 else 'orange' if x > 20 else 'green' 
                         for x in df_geography['churn_rate']],
            name='Churn Rate'
        ),
        row=1, col=1
    )
    
    # Pie chart: Customer distribution
    fig.add_trace(
        go.Pie(
            labels=df_geography['geography'],
            values=df_geography['total_customers'],
            hole=0.3
        ),
        row=1, col=2
    )
    
    fig.update_xaxes(title_text="Geography", row=1, col=1)
    fig.update_yaxes(title_text="Churn Rate (%)", row=1, col=1)
    
    fig.update_layout(
        height=500,
        title_text="Geographic Churn Analysis (Last 30 Days)",
        showlegend=False
    )
    
    fig.show()
else:
    print("⏳ Waiting for geographic data...")


🌍 Geography-wise Analysis:


,geography,total_customers,churn_count,churn_rate,avg_risk_score,avg_age,avg_balance
0,France,943,311,32.98,0.652,38.9,62534.62
1,Spain,477,141,29.56,0.621,38.5,61033.74
2,Germany,439,32,7.29,0.665,39.2,120933.57


## 🚨 3. High-Risk Customer Analysis


In [7]:
# Query top high-risk customers
query_high_risk = """
    SELECT * FROM v_top_risk_customers
    ORDER BY max_risk_score DESC
    LIMIT 20;
"""

df_high_risk = query_to_dataframe(query_high_risk, conn)

if not df_high_risk.empty:
    print(f"🚨 Top {len(df_high_risk)} High-Risk Customers (Risk Score >= 0.7)")
    print(f"\n📊 Risk Distribution:")
    print(f"  • Extreme Risk (>0.9): {(df_high_risk['max_risk_score'] > 0.9).sum()}")
    print(f"  • High Risk (0.8-0.9): {((df_high_risk['max_risk_score'] >= 0.8) & (df_high_risk['max_risk_score'] <= 0.9)).sum()}")
    print(f"  • Moderate Risk (0.7-0.8): {((df_high_risk['max_risk_score'] >= 0.7) & (df_high_risk['max_risk_score'] < 0.8)).sum()}")
    
    # Display top 10
    display(df_high_risk.head(10))
    
    # Risk score distribution
    fig = go.Figure()
    
    fig.add_trace(go.Histogram(
        x=df_high_risk['max_risk_score'],
        nbinsx=20,
        marker_color='red',
        opacity=0.7
    ))
    
    fig.update_layout(
        title="High-Risk Customer Distribution",
        xaxis_title="Risk Score",
        yaxis_title="Number of Customers",
        height=400
    )
    
    fig.show()
    
    # Age vs Balance for high-risk customers
    fig2 = px.scatter(
        df_high_risk,
        x='age',
        y='balance',
        size='max_risk_score',
        color='geography',
        hover_data=['customer_id', 'max_risk_score'],
        title="High-Risk Customers: Age vs Balance",
        labels={'age': 'Age', 'balance': 'Account Balance'},
        height=500
    )
    
    fig2.show()
else:
    print("⏳ Waiting for high-risk customer data...")


🚨 Top 20 High-Risk Customers (Risk Score >= 0.7)

📊 Risk Distribution:
  • Extreme Risk (>0.9): 0
  • High Risk (0.8-0.9): 20
  • Moderate Risk (0.7-0.8): 0


,customer_id,max_risk_score,last_prediction,geography,gender,age,balance,tenure
0,15771483,0.8804,2025-10-18 18:41:06.131410,France,Male,40,0.00,6
1,15650454,0.8798,2025-10-18 18:47:41.816375,France,Male,57,0.00,5
2,15698246,0.8786,2025-10-18 18:47:41.689545,France,Female,24,0.00,2
3,15734311,0.8785,2025-10-18 18:42:46.522711,France,Female,27,0.00,3
4,15626156,0.8765,2025-10-18 18:40:03.123426,France,Female,60,0.00,3
5,15798906,0.8751,2025-10-18 18:47:52.697793,France,Male,69,0.00,5
6,15631693,0.8735,2025-10-18 18:47:08.726815,France,Male,36,0.00,7
7,15588928,0.8731,2025-10-18 18:45:41.772185,France,Male,47,0.00,5
8,15749947,0.8727,2025-10-18 18:45:19.528842,France,Female,44,0.00,7
9,15725509,0.8725,2025-10-18 18:46:28.043583,Germany,Male,30,115286.68,5


## 📈 4. Churn Trends Over Time


In [8]:
# Query churn trends
query_trends = """
    SELECT 
        date,
        SUM(predictions) as total_predictions,
        SUM(churns) as total_churns,
        AVG(churn_rate) as avg_churn_rate,
        AVG(avg_risk) as avg_risk_score
    FROM v_churn_trends
    GROUP BY date
    ORDER BY date DESC
    LIMIT 30;
"""

df_trends = query_to_dataframe(query_trends, conn)

if not df_trends.empty:
    print(f"📅 Trend data: {len(df_trends)} days")
    
    # Reverse for chronological order
    df_trends = df_trends.sort_values('date')
    
    # Create trend visualization
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Daily Prediction Volume', 'Churn Rate Trend'),
        vertical_spacing=0.15
    )
    
    # Daily predictions
    fig.add_trace(
        go.Bar(
            x=df_trends['date'],
            y=df_trends['total_predictions'],
            name='Predictions',
            marker_color='lightblue'
        ),
        row=1, col=1
    )
    
    # Churn rate trend with moving average
    fig.add_trace(
        go.Scatter(
            x=df_trends['date'],
            y=df_trends['avg_churn_rate'],
            name='Churn Rate',
            mode='lines+markers',
            line=dict(color='red', width=2)
        ),
        row=2, col=1
    )
    
    # Add 7-day moving average
    if len(df_trends) >= 7:
        df_trends['ma7'] = df_trends['avg_churn_rate'].rolling(window=7).mean()
        fig.add_trace(
            go.Scatter(
                x=df_trends['date'],
                y=df_trends['ma7'],
                name='7-Day MA',
                mode='lines',
                line=dict(color='darkred', width=3, dash='dash')
            ),
            row=2, col=1
        )
    
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Count", row=1, col=1)
    fig.update_yaxes(title_text="Churn Rate (%)", row=2, col=1)
    
    fig.update_layout(
        height=700,
        title_text="Churn Prediction Trends (Last 30 Days)"
    )
    
    fig.show()
else:
    print("⏳ Waiting for trend data...")


📅 Trend data: 1 days


## 🎯 5. Model Performance Metrics


In [9]:
# Query model performance
query_performance = """
    SELECT * FROM v_model_performance
    ORDER BY date DESC
    LIMIT 30;
"""

df_performance = query_to_dataframe(query_performance, conn)

if not df_performance.empty:
    print("🎯 Model Performance Metrics:")
    print(f"\n📊 Confidence Analysis:")
    print(f"  • Average Confidence: {df_performance['avg_confidence'].mean():.3f}")
    print(f"  • High Confidence (>0.8): {df_performance['high_confidence_count'].sum():,.0f} predictions")
    print(f"  • Low Confidence (<0.6): {df_performance['low_confidence_count'].sum():,.0f} predictions")
    
    display(df_performance.head(10))
    
    # Confidence distribution over time
    df_performance_sorted = df_performance.sort_values('date')
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df_performance_sorted['date'],
        y=df_performance_sorted['avg_confidence'],
        name='Average Confidence',
        mode='lines+markers',
        line=dict(color='blue', width=3)
    ))
    
    # Add confidence threshold lines
    fig.add_hline(y=0.8, line_dash="dash", line_color="green", 
                  annotation_text="High Confidence Threshold")
    fig.add_hline(y=0.6, line_dash="dash", line_color="orange", 
                  annotation_text="Low Confidence Threshold")
    
    fig.update_layout(
        title="Model Confidence Over Time",
        xaxis_title="Date",
        yaxis_title="Confidence Score",
        height=500
    )
    
    fig.show()
else:
    print("⏳ Waiting for model performance data...")


🎯 Model Performance Metrics:

📊 Confidence Analysis:
  • Average Confidence: 0.399
  • High Confidence (>0.8): 0 predictions
  • Low Confidence (<0.6): 1,655 predictions


,model_version,date,total_predictions,avg_confidence,high_confidence_count,low_confidence_count
0,sklearn_v1.0,2025-10-18,1859,0.399,0,1655


## 📊 6. Overall Statistics & Summary


In [10]:
# Get overall statistics from churn_predictions table
query_overall = """
    SELECT 
        COUNT(*) as total_predictions,
        SUM(prediction) as total_churns,
        ROUND(SUM(prediction)::numeric / COUNT(*)::numeric * 100, 2) as overall_churn_rate,
        ROUND(AVG(probability)::numeric, 3) as avg_probability,
        ROUND(AVG(risk_score)::numeric, 3) as avg_risk_score,
        SUM(CASE WHEN risk_score >= 0.7 THEN 1 ELSE 0 END) as high_risk_count,
        MIN(predicted_at) as first_prediction,
        MAX(predicted_at) as last_prediction,
        COUNT(DISTINCT customer_id) as unique_customers,
        COUNT(DISTINCT geography) as geographies_covered
    FROM churn_predictions;
"""

df_overall = query_to_dataframe(query_overall, conn)

if not df_overall.empty and df_overall['total_predictions'].iloc[0] > 0:
    stats = df_overall.iloc[0]
    
    print("="*70)
    print("📊 OVERALL CHURN PREDICTION ANALYTICS SUMMARY")
    print("="*70)
    print(f"\n⏰ Time Period:")
    print(f"   First Prediction: {stats['first_prediction']}")
    print(f"   Last Prediction:  {stats['last_prediction']}")
    
    print(f"\n📈 Prediction Metrics:")
    print(f"   Total Predictions:    {stats['total_predictions']:>10,}")
    print(f"   Unique Customers:     {stats['unique_customers']:>10,}")
    print(f"   Predicted Churns:     {stats['total_churns']:>10,}")
    print(f"   Overall Churn Rate:   {stats['overall_churn_rate']:>10.2f}%")
    
    print(f"\n🎯 Model Performance:")
    print(f"   Average Probability:  {stats['avg_probability']:>10.3f}")
    print(f"   Average Risk Score:   {stats['avg_risk_score']:>10.3f}")
    
    print(f"\n🚨 Risk Analysis:")
    print(f"   High-Risk Customers:  {stats['high_risk_count']:>10,}")
    print(f"   High-Risk Rate:       {(stats['high_risk_count']/stats['total_predictions']*100):>10.2f}%")
    
    print(f"\n🌍 Coverage:")
    print(f"   Geographies Covered:  {stats['geographies_covered']:>10}")
    print("="*70)
    
    # Create summary visualization
    fig = go.Figure()
    
    fig.add_trace(go.Indicator(
        mode="number+delta",
        value=stats['total_predictions'],
        title={"text": "Total Predictions"},
        domain={'x': [0, 0.25], 'y': [0.7, 1]}
    ))
    
    fig.add_trace(go.Indicator(
        mode="number+gauge",
        value=stats['overall_churn_rate'],
        title={"text": "Churn Rate (%)"},
        gauge={'axis': {'range': [0, 100]},
               'bar': {'color': "red"},
               'threshold': {'line': {'color': "orange", 'width': 4},
                           'thickness': 0.75, 'value': 20}},
        domain={'x': [0.3, 0.7], 'y': [0.5, 1]}
    ))
    
    fig.add_trace(go.Indicator(
        mode="number",
        value=stats['high_risk_count'],
        title={"text": "High-Risk Customers"},
        domain={'x': [0.75, 1], 'y': [0.7, 1]}
    ))
    
    fig.add_trace(go.Indicator(
        mode="number+gauge",
        value=stats['avg_probability'],
        title={"text": "Avg Confidence"},
        gauge={'axis': {'range': [0, 1]},
               'bar': {'color': "blue"}},
        domain={'x': [0, 0.5], 'y': [0, 0.4]}
    ))
    
    fig.add_trace(go.Indicator(
        mode="number+gauge",
        value=stats['avg_risk_score'],
        title={"text": "Avg Risk Score"},
        gauge={'axis': {'range': [0, 1]},
               'bar': {'color': "orange"}},
        domain={'x': [0.5, 1], 'y': [0, 0.4]}
    ))
    
    fig.update_layout(
        title="Real-Time Churn Analytics Dashboard",
        height=800
    )
    
    fig.show()
else:
    print("⏳ No prediction data available yet.")
    print("\n💡 To start collecting data:")
    print("   1. Ensure Kafka producer is streaming events")
    print("   2. Ensure Kafka consumer is making predictions")
    print("   3. Wait a few minutes for predictions to accumulate")
    print("   4. Re-run this notebook")


📊 OVERALL CHURN PREDICTION ANALYTICS SUMMARY

⏰ Time Period:
   First Prediction: 2025-10-18 18:39:58.124756
   Last Prediction:  2025-10-18 18:48:49.321052

📈 Prediction Metrics:
   Total Predictions:         1,859
   Unique Customers:          1,691
   Predicted Churns:            484
   Overall Churn Rate:        26.04%

🎯 Model Performance:
   Average Probability:       0.399
   Average Risk Score:        0.647

🚨 Risk Analysis:
   High-Risk Customers:         507
   High-Risk Rate:            27.27%

🌍 Coverage:
   Geographies Covered:           3


# Check all service status
docker ps --format "table {{.Names}}\t{{.Status}}" | grep -E "kafka-|NAMES"

# Watch producer streaming events
docker logs kafka-producer --follow

# Watch consumer making predictions (will show once model loads)
docker logs kafka-consumer --follow

# Watch analytics storing to RDS
docker logs kafka-analytics --follow

# Check Kafka UI
open http://localhost:8090

In [11]:
# Query customer demographics
query_demographics = """
    SELECT 
        gender,
        geography,
        COUNT(*) as count,
        SUM(prediction) as churns,
        ROUND(SUM(prediction)::numeric / COUNT(*)::numeric * 100, 2) as churn_rate,
        ROUND(AVG(age)::numeric, 1) as avg_age,
        ROUND(AVG(balance)::numeric, 2) as avg_balance,
        ROUND(AVG(tenure)::numeric, 1) as avg_tenure
    FROM churn_predictions
    GROUP BY gender, geography
    ORDER BY churn_rate DESC;
"""

df_demographics = query_to_dataframe(query_demographics, conn)

if not df_demographics.empty:
    print("👥 Customer Demographics Analysis:")
    display(df_demographics)
    
    # Create heatmap of churn rates
    pivot_churn = df_demographics.pivot(index='geography', columns='gender', values='churn_rate')
    
    fig = go.Figure(data=go.Heatmap(
        z=pivot_churn.values,
        x=pivot_churn.columns,
        y=pivot_churn.index,
        colorscale='Reds',
        text=pivot_churn.values,
        texttemplate='%{text:.1f}%',
        textfont={"size": 14}
    ))
    
    fig.update_layout(
        title="Churn Rate Heatmap: Geography vs Gender",
        xaxis_title="Gender",
        yaxis_title="Geography",
        height=400
    )
    
    fig.show()
else:
    print("⏳ Waiting for demographic data...")


👥 Customer Demographics Analysis:


,gender,geography,count,churns,churn_rate,avg_age,avg_balance,avg_tenure
0,Female,France,424,164,38.68,38.3,60644.98,5.0
1,Female,Spain,214,72,33.64,38.2,59597.24,4.9
2,Male,France,519,147,28.32,39.4,64078.37,5.0
3,Male,Spain,263,69,26.24,38.7,62202.61,4.8
4,Female,Germany,201,20,9.95,39.3,119370.41,4.9
5,Male,Germany,238,12,5.04,39.2,122253.72,4.8


## 🔄 8. Recent Predictions Stream


In [12]:
# Query most recent predictions
query_recent = """
    SELECT 
        customer_id,
        CASE WHEN prediction = 1 THEN 'CHURN' ELSE 'RETAIN' END as prediction_label,
        ROUND(probability::numeric, 3) as probability,
        ROUND(risk_score::numeric, 3) as risk_score,
        geography,
        gender,
        age,
        balance,
        tenure,
        predicted_at
    FROM churn_predictions
    ORDER BY predicted_at DESC
    LIMIT 50;
"""

df_recent = query_to_dataframe(query_recent, conn)

if not df_recent.empty:
    print(f"🔄 Most Recent {len(df_recent)} Predictions:")
    
    # Style the dataframe
    def highlight_churn(row):
        if row['prediction_label'] == 'CHURN':
            return ['background-color: #ffcccc']*len(row)
        else:
            return ['background-color: #ccffcc']*len(row)
    
    styled_df = df_recent.head(20).style.apply(highlight_churn, axis=1)
    display(styled_df)
else:
    print("⏳ Waiting for predictions...")


🔄 Most Recent 50 Predictions:


,customer_id,prediction_label,probability,risk_score,geography,gender,age,balance,tenure,predicted_at
0,15584477,RETAIN,0.426000,0.574000,Spain,Female,35,106405.030000,1,2025-10-18 18:48:49.321052
1,15806956,CHURN,0.636000,0.636000,Spain,Male,30,112666.670000,1,2025-10-18 18:48:49.196372
2,15588944,CHURN,0.586000,0.586000,France,Female,63,165350.610000,1,2025-10-18 18:48:49.062866
3,15694272,RETAIN,0.383000,0.617000,France,Male,30,64097.750000,1,2025-10-18 18:48:48.559994
4,15797227,RETAIN,0.185000,0.816000,France,Male,28,0.000000,8,2025-10-18 18:48:48.434163
5,15621550,CHURN,0.758000,0.758000,Spain,Female,50,140292.580000,1,2025-10-18 18:48:48.293082
6,15815530,CHURN,0.612000,0.612000,France,Female,42,75497.510000,10,2025-10-18 18:48:48.160811
7,15754168,RETAIN,0.325000,0.675000,France,Female,40,0.000000,3,2025-10-18 18:48:48.026208
8,15793307,CHURN,0.724000,0.724000,Spain,Female,41,142880.280000,4,2025-10-18 18:48:47.847266
9,15633141,RETAIN,0.411000,0.589000,Germany,Female,35,174902.260000,4,2025-10-18 18:48:47.249685


## 🔍 9. Data Quality & Health Check


In [13]:
# Check data quality metrics
print("🔍 Database Health Check:")
print("="*70)

tables = ['churn_predictions', 'churn_metrics_hourly', 'churn_metrics_daily', 'high_risk_customers']

for table in tables:
    try:
        stats = get_table_stats(table, conn)
        if not stats.empty:
            print(f"\n📊 {table.upper()}:")
            print(f"   Total Rows:    {stats['total_rows'].iloc[0]:>10,}")
            if stats['total_rows'].iloc[0] > 0:
                print(f"   Oldest Record: {stats['oldest_record'].iloc[0]}")
                print(f"   Newest Record: {stats['newest_record'].iloc[0]}")
    except Exception as e:
        print(f"\n⚠️ {table}: {str(e)[:100]}")

print("\n" + "="*70)


🔍 Database Health Check:

📊 CHURN_PREDICTIONS:
   Total Rows:         1,859
   Oldest Record: 2025-10-18 18:39:58.380181
   Newest Record: 2025-10-18 18:48:49.581235

📊 CHURN_METRICS_HOURLY:
   Total Rows:             0

📊 CHURN_METRICS_DAILY:
   Total Rows:             0
❌ Query failed: Execution failed on sql '
        SELECT 
            COUNT(*) as total_rows,
            MIN(created_at) as oldest_record,
            MAX(created_at) as newest_record
        FROM high_risk_customers;
    ': column "created_at" does not exist
LINE 4:             MIN(created_at) as oldest_record,
                        ^




## 💾 10. Export Analytics Report


In [14]:
# Export summary report to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
export_dir = "../artifacts/analytics_reports"
os.makedirs(export_dir, exist_ok=True)

# Export dataframes if they exist
exports = [
    (df_overall, "overall_summary"),
    (df_realtime, "realtime_dashboard"),
    (df_geography, "geography_analysis"),
    (df_high_risk, "high_risk_customers"),
    (df_trends, "churn_trends"),
    (df_performance, "model_performance"),
    (df_demographics, "demographics"),
    (df_recent, "recent_predictions")
]

exported_files = []
for df, name in exports:
    if df is not None and not df.empty:
        filename = f"{export_dir}/{name}_{timestamp}.csv"
        df.to_csv(filename, index=False)
        exported_files.append(filename)
        print(f"✅ Exported: {filename}")

if exported_files:
    print(f"\n📁 Total {len(exported_files)} reports exported to: {export_dir}")
else:
    print("⏳ No data available to export yet")


✅ Exported: ../artifacts/analytics_reports/overall_summary_20251019_002052.csv
✅ Exported: ../artifacts/analytics_reports/realtime_dashboard_20251019_002052.csv
✅ Exported: ../artifacts/analytics_reports/geography_analysis_20251019_002052.csv
✅ Exported: ../artifacts/analytics_reports/high_risk_customers_20251019_002052.csv
✅ Exported: ../artifacts/analytics_reports/churn_trends_20251019_002052.csv
✅ Exported: ../artifacts/analytics_reports/model_performance_20251019_002052.csv
✅ Exported: ../artifacts/analytics_reports/demographics_20251019_002052.csv
✅ Exported: ../artifacts/analytics_reports/recent_predictions_20251019_002052.csv

📁 Total 8 reports exported to: ../artifacts/analytics_reports


## 🔚 Close Database Connection


In [15]:
# Close connection
if conn:
    conn.close()
    print("✅ Database connection closed")


✅ Database connection closed


---

## 📝 Notes

**Data Pipeline:**
1. Kafka Producer → Streams customer events (10 events/second)
2. Kafka Consumer → Makes real-time churn predictions
3. Analytics Service → Stores predictions in RDS PostgreSQL
4. This Notebook → Visualizes analytics from RDS

**Refresh Frequency:**
- Re-run this notebook to see updated analytics
- Data updates in real-time as predictions are made
- Aggregated views (hourly/daily) update automatically via database triggers

**Next Steps:**
- Connect to AWS QuickSight for real-time dashboards
- Set up automated email alerts for high-risk customers
- Schedule this notebook to run periodically for reports

---
